# Setup

In [ ]:
try:
    import dowhy
except:
    # install libraries
    !pip install bertopic dowhy numba numpy datasets
    !pip uninstall --yes numpy
    !pip uninstall --yes numba
    !pip install numba

# Dataset

We'll analyze the [IMDB large movie reviews dataset](https://ai.stanford.edu/~amaas/data/sentiment/), which was introduced in Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).

In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("imdb")
ds

In [ ]:
import re
import string
 

def clean(text):
    # Remove line breaks
    text = re.sub(r"<br />", "", text)
    return text

clean_ds = ds.map(lambda example: {"text": clean(example["text"])})

In [ ]:
clean_ds["train"]["text"][0]

# Causal Inference

Question: What influence do specific topics have on the sentiment of a movie review?

* Treatment variable: A specific topic
* Outcome variables: sentiment of review
* Confounders/covariates: embedding of the review text

The treatment variable will be binary and corresponds to the presence or absence of specific topics in the movie review, and the outcome variable will be the sentiment of the review. The sentence embeddings of the reviews will be used as covariates to control for potential confounding factors.

We will compute the Conditional Average Treatment Effect (CATE) to get the causal effect a specific topic has on the sentiment of the movie reviews. To compute the CATE, we will condition on the covariates.

## Embeddings

Sentence embeddings are used as the covariates and as input to the  BERTopic topic modeling process, so we will compute them first.

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

In [ ]:
train_embeddings = sentence_model.encode(clean_ds["train"]["text"], show_progress_bar=True, device="cuda")
test_embeddings = sentence_model.encode(clean_ds["test"]["text"], show_progress_bar=True, device="cuda")

 ## Topics

 We need to compute the topics so we can use them as treatments. We'll use BERTopic for this, as there are minimal parameters to tune and the results are of high quality.

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))

In [ ]:
train_docs = list(clean_ds["train"]["text"])
test_docs = list(clean_ds["test"]["text"])

In [ ]:
NUM_TOPICS = 50

topic_model = BERTopic(
    embedding_model=sentence_model, 
    umap_model=umap_model, 
    vectorizer_model=vectorizer_model,
    calculate_probabilities=True,
    nr_topics=NUM_TOPICS,
)

topics, probs = topic_model.fit_transform(train_docs, train_embeddings)

In [ ]:
# fine-tune topic representation
vectorizer_model_1 = CountVectorizer(stop_words="english", ngram_range=(1, 3), min_df=10)
topic_model.update_topics(train_docs, vectorizer_model=vectorizer_model_1)

### Visualize Topics

In [ ]:
# topic_model.visualize_topics()

### Visualize Documents

Reduce embeddings to 2D specifically for plotting.

In [ ]:
umap_2d = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine')
train_embeddings_2d = umap_2d.fit_transform(train_embeddings, clean_ds["train"]["label"])

In [ ]:
topic_model.visualize_documents(
    train_docs, 
    reduced_embeddings=train_embeddings_2d, 
    hide_document_hover=False, 
    hide_annotations=True,
)

### Visualize topic distribution

This will show how each token in a review contributes to the topics to which the review is assigned.

We need soft clusters to more than one topic assigned to a movie overview.

In [ ]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(
    train_docs,
    calculate_tokens=True,
    window=4,
)

In [ ]:
df = topic_model.visualize_approximate_distribution(train_docs[0], topic_token_distr[0])
df

In [ ]:
df = topic_model.visualize_approximate_distribution(train_docs[134], topic_token_distr[134])
df

### Topic labels

This will help when we select which topic to use a treatment.

In [ ]:
topic_labels = topic_model.generate_topic_labels(
    nr_words=4,
    topic_prefix=True,
    word_length=10,
    separator="_",
)

id2topic = {i-1: f"t_{label}" for i, label in enumerate(topic_labels)}

# drop outlier topic
del id2topic[-1]

In [ ]:
id2topic[0]

Now that we have topics, lets add them to the dataset as columns. The shape of the topic distribution is shown below. First, we convert each split into a dataframe

We also need to generate the topic for the test set.

In [ ]:
topic_distr.shape

In [ ]:
clean_ds

In [ ]:
test_topic_distr, test_topic_token_distr = topic_model.approximate_distribution(
    test_docs,
    calculate_tokens=True,
    window=4,
)

In [ ]:
df = topic_model.visualize_approximate_distribution(test_docs[134], test_topic_token_distr[134])
df

We will add one column to each dataset split for each topic.

In [ ]:
import pandas as pd

topic_names = [id2topic[i] for i in range(topic_distr.shape[1])]
train_topics_df = pd.DataFrame(data=topic_distr, columns=topic_names)
train_topics_df.head()

In [ ]:
test_topics_df = pd.DataFrame(data=test_topic_distr, columns=topic_names)
test_topics_df.head()

In [ ]:
train_df = clean_ds["train"].to_pandas()

train_df_full = pd.concat([train_df, train_topics_df], axis=1)
print(train_df_full.shape)
train_df_full.head()

In [ ]:
test_df = clean_ds["test"].to_pandas()

test_df_full = pd.concat([test_df, test_topics_df], axis=1)
print(test_df_full.shape)
test_df_full.head()

## Covariates

We'll use the same sentence embeddings computed above. As shown below, the embedding dimension is 384, which is rather high, so we will use UMAP to reduce the dimension.

In [ ]:
train_embeddings.shape

In [ ]:
n_covariates = 20

embedding_column_names = [f"e_{i:04d}" for i in range(n_covariates)]

umap_cov = UMAP(n_neighbors=10, n_components=n_covariates, min_dist=0.0, metric='cosine')
train_covariates = umap_cov.fit_transform(train_embeddings)
test_covariates = umap_cov.transform(test_embeddings)

In [ ]:
print(train_covariates.shape)
print(test_covariates.shape)

Now we add the covariates to the dataframes.

In [ ]:
X_df = pd.DataFrame(data=train_covariates, columns=embedding_column_names)
train_df_all = pd.concat([train_df_full, X_df], axis=1)
print(train_df_all.shape)
train_df_all.head()

In [ ]:
X_df = pd.DataFrame(data=test_covariates, columns=embedding_column_names)
test_df_all = pd.concat([test_df_full, X_df], axis=1)
print(test_df_all.shape)
test_df_all.head()

### Outcome columns

Our outcome variable is the sentiment label. Positive sentiment is label = 1, negative sentiment is label = 0, so we need not binarize the labels.

In [ ]:
def get_outcome_values(df):
    Y = df["label"].to_numpy()
    return Y

### Treatment data

Grab binarized treatment values

In [ ]:
def get_treatment_values(df, topic_id, threshold=0.5):
    column_name = id2topic[topic_id]
    T = df[column_name].apply(lambda x: x >= threshold).astype(int).to_numpy()
    return T

### Covariate columns

Grab covariates.

In [ ]:
def get_covariate_values(df):
    X = df[embedding_column_names].to_numpy()
    return X

## Meta-learners

We'll pick a topic to use as treatment. All topics are listed in the cell below.

In [ ]:
for i, name in id2topic.items():
    print(i, name)

In [ ]:
n = 48
print(f"Topic: {id2topic[n]}\n")

for (word, score) in topic_model.get_topic(n):
    print(word, score)

print(train_df_all[ train_df_all[id2topic[n]] >= 0.01 ].label.value_counts())
train_df_all[ train_df_all[id2topic[n]] >= 0.01 ]

In [ ]:
n = 27
print(f"Topic: {id2topic[n]}\n")

for (word, score) in topic_model.get_topic(n):
    print(word, score)

print(train_df_all[ train_df_all[id2topic[n]] >= 0.1 ].label.value_counts())
train_df_all[ train_df_all[id2topic[n]] >= 0.1 ]

In [ ]:
# treatment
topic_id = 27  # star wars

# picking a threshold of 0.1 to consider the treatment as present
T_train = get_treatment_values(train_df_all, topic_id, threshold=0.1)
T_test = get_treatment_values(test_df_all, topic_id, threshold=0.1)
print(f"T_train: {T_train.shape}")
print(f"T_test: {T_test.shape}")

Y_train = get_outcome_values(train_df_all)
Y_test = get_outcome_values(test_df_all)
print(f"Y_train: {Y_train.shape}")
print(f"Y_test: {Y_test.shape}")

X_train = get_covariate_values(train_df_all)
print(f"X_train: {X_train.shape}")

X_test = get_covariate_values(test_df_all)
print(f"X_test: {X_test.shape}")

## Train meta-learner causal estimators

In [ ]:
# Main imports
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner

# Helper imports
import numpy as np
from numpy.random import binomial, multivariate_normal, normal, uniform
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
min_samples_leaf = len(T_train) // 100

### T-learner

In [ ]:
models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
T_learner = TLearner(models=models)

T_learner.fit(Y_train, T_train, X=X_train)

# Estimate treatment effects on test data
T_te = T_learner.effect(X_test)

### S-learner

In [ ]:
overall_model = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
S_learner = SLearner(overall_model=overall_model)

S_learner.fit(Y_train, T_train, X=X_train)

# Estimate treatment effects on test data
S_te = S_learner.effect(X_test)

### X-learner

In [ ]:
# Instantiate X learner
models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
propensity_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=6,
    min_samples_leaf=min_samples_leaf
)

X_learner = XLearner(models=models, propensity_model=propensity_model)
X_learner.fit(Y_train, T_train, X=X_train)

# Estimate treatment effects on test data
X_te = X_learner.effect(X_test)

## Comparing treatment effects from different meta-learners

In [ ]:
plt.figure(figsize=(7, 5))
x_axis = list(range(len(X_test)))

plt.scatter(x_axis, T_te, label="T-learner", alpha=0.5)
plt.scatter(x_axis, S_te, label="S-learner", alpha=0.5)
plt.scatter(x_axis, X_te, label="X-learner", alpha=0.5)

plt.xlabel('Movie Review id')
plt.ylabel('Treatment Effect')
plt.title(f"Treatment Topic: {id2topic[topic_id]}")
plt.legend()
plt.show()

Let's look at the distribution CATE values for any topic and for each of the meta-learners used above.

We'll plot the distribution of CATE values for several topics and topic score thresholds.

In [ ]:
def run_analysis(topic_id, topic_threshold=0.1):
    # topic_id is treatment
    # picking a threshold of 0.1 to consider the treatment as present
    T_train = get_treatment_values(train_df_all, topic_id, threshold=topic_threshold)
    T_test = get_treatment_values(test_df_all, topic_id, threshold=topic_threshold)

    treated_indices = np.where(T_test == 1)[0]
    untreated_indices = np.where(T_test == 0)[0]
    
    Y_train = get_outcome_values(train_df_all)
    Y_test = get_outcome_values(test_df_all)
    
    X_train = get_covariate_values(train_df_all)
    X_test = get_covariate_values(test_df_all)
    
    print("Building T-Learner...")
    models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
    T_learner = TLearner(models=models)

    T_learner.fit(Y_train, T_train, X=X_train)
    T_te = T_learner.effect(X_test)
    
    print("Building S-Learner...")
    overall_model = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
    S_learner = SLearner(overall_model=overall_model)

    S_learner.fit(Y_train, T_train, X=X_train)
    S_te = S_learner.effect(X_test)
    
    print("Building X-Learner...")
    models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=min_samples_leaf)
    propensity_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=6,
        min_samples_leaf=min_samples_leaf
    )

    X_learner = XLearner(models=models, propensity_model=propensity_model)
    X_learner.fit(Y_train, T_train, X=X_train)
    X_te = X_learner.effect(X_test)
    
    plt.figure(figsize=(7, 5))
    x_axis = np.arange(len(X_test))
    
    plt.hist(T_te, label="T-learner", bins=10, alpha=0.5)
    plt.hist(S_te, label="S-learner", bins=10, alpha=0.5)
    plt.hist(X_te, label="X-learner", bins=10, alpha=0.5)
    
    plt.xlabel('Treatment Effect')
    plt.ylabel('Frequency')
    plt.title(f"Treatment Topic: {id2topic[topic_id]}")

    
    # plt.scatter(x_axis, T_te, label="T-learner")
    # plt.scatter(x_axis, S_te, label="S-learner")
    # plt.scatter(x_axis, X_te, label="X-learner")
    
    # plt.xlabel('Movie Review id')
    # plt.ylabel('Treatment Effect')
    # plt.title(f"Treatment Topic: {id2topic[topic_id]}")
    plt.legend()
    plt.show()

In [ ]:
run_analysis(1, topic_threshold=0.1)

In [ ]:
run_analysis(27, topic_threshold=0.1)

In [ ]:
run_analysis(15, topic_threshold=0.1)

In [ ]:
run_analysis(9, topic_threshold=0.1)

In [ ]:
run_analysis(23, topic_threshold=0.1)

# Analysis

So, what does the distribution of CATE scores tell us?

Generally speaking, when the outcome variable is binary-valued (as it is for our movie review data), a negative CATE value means that the treatment has a negative/detrimental effect on the outcome variable. For the IMDB data, that means that when the topic is present in the movie review, said review is more likely to be negative than similar reviews without the topic. Similarly, a positive CATE value suggests that the topic tends to make the review more positive.

## S-Learner CATE scores

A few of the plots have a large peak for the S-Learner that is near zero. The S-Learner scores are then suggesting that each topic has a negligible effect of the sentiment of movie reviews. 

However, the S-Learner assumes that the treatment effect is homogeneous across the population. If that assumption is not correct (and we have no reason to think it should be), then treatment effect is not homogeneous across the population, then the CATE scores calculated using an S-Learner may not accurately reflect the true treatment effect. 

## T-Learner and X-Learner CATE scores

The histograms for the T-Learner and X-Learner suggest that the effect a given topic has on the movie review sentiment is sometimes positive and sometimes negative. The first plot above shows that we can actually see which specific movie reviews have positive and negative scores.

We can also note that the some topics appear to have a large effect on sentiment, for example, topic 23, the Dan Brown topic, as shown in the plot immediately above.

